# Introduction #

In the ever-evolving landscape of financial markets, data-driven decision-making is becoming increasingly essential [source]. The integration of artificial intelligence (AI) into finance, particularly through the use of machine learning models, has shown significant potential in optimizing investment strategies [source]. However, the complexity of financial markets requires more than just individual models. An ensemble approach— leveraging multiple models to assess financial, economic, and sentiment data —may provide a more comprehensive and accurate view of market opportunities.

This project aims to explore the design and development of a hybrid ensemble AI system for investment analysis. By combining technical, fundamental, economic, and sentiment-based models, the system seeks to generate reliable buy and sell signals only when a consensus is reached. The goal is to enhance accuracy and profitability in trading decisions by reducing noise and improving the reliability of predictions.

In addition to building a practical financial tool, this project serves as a research endeavor to deepen understanding of AI ensemble methods, their application in finance, and their potential to outperform individual models or traditional strategies.

## Objectives ##

To design and implement a hybrid ensemble AI system that integrates multiple financial analysis models—technical, fundamental, economic, and sentiment-based—to provide data-driven investment signals. By requiring a threshold of consensus among models through the use of weighted voting, the system aims to improve decision-making accuracy and profitability in trading strategies.

## Research & Learning Objectives ##

To deepen the understanding of ensemble AI methods in financial markets, exploring the interaction between different model types and refining techniques for model integration, consensus mechanisms, and trading strategy optimization. This project also serves as a learning platform to enhance practical skills in AI development, evaluation, and deployment within the context of algorithmic trading.

# Methodology / Approach #

The primary focus of this project is on the AI-driven decision-making process rather than the mechanics of trading bot development. Therefore, we will leverage established trading bot libraries that allow for the easy definition and implementation of various trading strategies. Using these libraries provides several advantages:
- Ease of use: They are user-friendly and provide abstractions that allow us to focus on the AI aspects.
- Time-efficient: Utilizing pre-built libraries reduces the time needed to develop a functioning backtesting system.
- Project focus: Since the goal of this project is to experiment with and improve AI models, rather than creating a custom trading bot, these libraries allow us to dedicate more resources to the AI components.

Each model within the ensemble will be designed using modern AI techniques (e.g., deep learning, reinforcement learning, or hybrid models) and will be trained to function independently. The performance of each model will be benchmarked against established financial indices such as the S&P 500 or NASDAQ 100, ensuring that each model performs at a level competitive with general market trends.

Each model will operate independently, utilizing distinct data sets related to the target security, which will encompass technical, fundamental, and sentiment-based features. Each model will provide a prediction in the form of a buy signal. It is important to note that the system is not tasked with identifying sell signals; the goal is to identify optimal buy opportunities, with the assumption that sell strategies may be handled by other components or human decision-makers.

Once each model provides its buy signal, the outputs will be aggregated using a weighted voting system. This system allows for models with higher accuracy or reliability to have a greater influence on the final decision. A predefined threshold will be applied, and if the aggregate signal exceeds the threshold, the system will generate a buy recommendation. The threshold is adjustable and can be fine-tuned during testing to balance risk and reward.


## Technologies ##

lorem ipsum

## Expected Outcomes ##

lorem ipsum




In [1]:
# Libraries


# ===================================
# Data Processing Imports
# ===================================
import pandas as pd
import matplotlib.pyplot as plt
#%matplotlib inline
import random

# ===================================
# Trading Bot Imports
# ===================================
import backtrader as bt
import yfinance as yf
from datetime import datetime


# ===================================
# AI Architecture Imports
# ===================================

## Trading Bots ##

The first step in this research is the implementation of several trading bots. Each bot is designed to operate based on well established but different trading strategies. 

Each bot will independently generate trading data, including buy/sell signals, performance metrics, and market conditions, and fundamentals of the particular security. This data will serve as the training dataset for its corresponding AI model within the ensemble. This approach allows for a diverse range of inputs and methodologies, enhancing the overall robustness and adaptability of the system.



### S&P 500 tickers for trading bots ###

In [2]:
#sp500_tickers = ['AAPL', 'TSLA']

# Scrape the S&P 500 data from Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
table = pd.read_html(url)  # Extract all tables from the URL

# The first table contains the S&P 500 data
sp500_table = table[0]

# Extract the 'Symbol' column as a list of ticker symbols
sp500_tickers = sp500_table['Symbol'].tolist()

# Sample a random number of tickers from the S&P 500
#random_tickers = random.sample(sp500_tickers, 50)

#sp500_tickers = random_tickers

### Trading Strategy: Momentum based investing

In this strategy the bot trades based off current momentum trends in the market. It buys when a stock has positive momentum over some perioud of time and then sells when that momentum turns negative. This strategy has performed well over recent years [SOURCE] and has some key advantages and disadvantages:

Advantages
- Trend Following: Momentum strategies capitalize on established trends, potentially leading to higher returns during bull markets.

- Behavioral Finance: They exploit behavioral biases, such as the tendency for investors to overreact to news or trends, allowing for profits from mispriced assets.

- Simplicity: Momentum strategies can be relatively straightforward to implement, using clear signals based on price movement.

- Historical Success: Many studies suggest that momentum strategies have historically outperformed the market over certain time frames.

- Diversification: Momentum can be applied across various asset classes (stocks, commodities, currencies), offering diversification benefits.

Disadvantages

- Reversals: Momentum can lead to significant losses if a trend reverses unexpectedly, as past performance does not guarantee future results.

- Market Volatility: During volatile or sideways markets, momentum strategies may struggle, leading to whipsaw losses.

- High Turnover: These strategies often require frequent trading, which can lead to higher transaction costs and tax implications.

- Risk of Herding: Momentum strategies can contribute to market bubbles as many traders chase the same trends, leading to instability when trends reverse.

- Lagging Indicators: Momentum indicators may lag in signaling the reversal of trends, causing late entry or exit points that reduce potential profits.



In [7]:
class MomentumStrategy(bt.Strategy):
    params = (
        ('buy_momentum_period', 10),
        ('sell_momentum_period', 5)     
    )

    def __init__(self):
        #dictionary of momentum indicators of each stock
        self.buy_momentum = {}
        self.sell_momentum = {}
        for i, data in enumerate(self.datas):
            self.buy_momentum[data] = bt.indicators.Momentum(data.close, period=self.params.buy_momentum_period)
            self.sell_momentum[data] = bt.indicators.Momentum(data.close, period=self.params.sell_momentum_period)
    
    def next(self):
        # Loop through each stock and make trading decisions:
        for i, data in enumerate(self.datas):
            #buy condition
            if self.buy_momentum[data][0] > 0 and not self.getposition(data):
                print(f"buying {data._name}")
                self.buy()
                
            #sell condition
            elif self.sell_momentum[data][0] < 0 and self.getposition(data):
                print(f"selling {data._name}")
                self.sell()

    

### Cerebro Engine ###
The cerebro engine will be used to produce backtesting results with the individual trading bots.

In [8]:
#initialize the engine
cerebro = bt.Cerebro()

for ticker in sp500_tickers:
    try:
        # Download data from Yahoo Finance
        df = yf.download(ticker, start='2023-01-01', end='2024-01-01')

        # Check if the DataFrame is empty or missing data
        if df.empty:
            print(f"Skipping {ticker}: No data available")
            continue
        
        # Prepare the data for Backtrader (make sure the index is in datetime format)
        df.index = pd.to_datetime(df.index)
        
        # Create a PandasData feed for Backtrader
        data = bt.feeds.PandasData(dataname=df)
        
        # Add the data to cerebro
        cerebro.adddata(data, name=ticker)
        print(f"Downloaded and added data for {ticker}")

    except Exception as e:
        print(f"ERROR for {ticker}: {e}")
    


[*********************100%***********************]  1 of 1 completed


Downloaded and added data for MMM


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Downloaded and added data for AOS
Downloaded and added data for ABT


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...

Downloaded and added data for XEL
Downloaded and added data for XYL
Downloaded and added data for YUM


[*********************100%***********************]  1 of 1 completed


Downloaded and added data for ZBRA


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Downloaded and added data for ZBH
Downloaded and added data for ZTS


In [5]:
#for data in cerebro.datas:
 #   print(data)

In [9]:
# Add the strategy
cerebro.addstrategy(MomentumStrategy)

# Set initial capital
cerebro.broker.setcash(100000)

    
# Run the backtest
cerebro.run()

# Plot results
#fig = cerebro.plot()
#plt.show(fig)

buying MMM
buying AOS
buying ABBV
buying ADBE
buying AES
buying AFL
buying AKAM
buying LNT
buying ALL
buying GOOGL
buying GOOG
buying MO
buying AMZN
buying AEE
buying AEP
buying AXP
buying AIG
buying AMT
buying AMP
buying AMGN
buying AON
buying APA
buying AAPL
buying AMAT
buying ACGL
buying ADM
buying ANET
buying AJG
buying AIZ
buying ATO
buying ADSK
buying ADP
buying AZO
buying AVB
buying AXON
buying BKR
buying BAC
buying BK
buying BBY
buying BIIB
buying BMY
buying AVGO
buying BG
buying BXP
buying CDNS
buying CPT
buying COF
buying CAH
buying CTLT
buying CBOE
buying CDW
buying COR
buying CNC
buying CNP
buying CF
buying SCHW
buying CHTR
buying CVX
buying CB
buying CI
buying CINF
buying CTAS
buying CSCO
buying C
buying CME
buying CMS
buying CL
buying CMCSA
buying CAG
buying COP
buying ED
buying CEG
buying COO
buying CPRT
buying CTVA
buying CSGP
buying COST
buying CTRA
buying CRWD
buying CCI
buying CVS
buying DAY
buying DECK
buying DE
buying DVN
buying FANG
buying DLR
buying DFS
buying DG

selling MMM
buying ADBE
buying AES
buying LNT
buying ALL
buying GOOGL
buying GOOG
buying MO
buying AEE
buying AEP
buying AMT
buying AMGN
buying APA
buying ACGL
buying AIZ
buying T
buying AVGO
buying CPB
buying CAH
buying CBOE
buying COR
buying CMG
buying CB
buying CHD
buying CI
buying CMS
buying KO
buying CL
buying CEG
buying COO
buying CTRA
buying DRI
buying DVN
buying DXCM
buying FANG
buying DG
buying DTE
buying EIX
buying EOG
buying EG
buying FAST
buying FSLR
buying FE
buying FI
buying GE
buying GIS
buying GILD
buying HES
buying HUM
buying IFF
buying KVUE
buying KMB
buying KMI
buying LVS
buying LLY
buying LMT
buying LULU
buying MRO
buying MKTX
buying MCD
buying MSFT
buying MDLZ
buying NFLX
buying NEM
buying NWSA
buying NWS
buying NEE
buying NKE
buying NOC
buying NRG
buying PCG
buying PNW
buying PPL
buying PG
buying PGR
buying PEG
buying RL
buying RTX
buying REG
buying ROST
buying SBAC
buying STX
buying SRE
buying SO
buying SBUX
buying SYY
buying TPR
buying TRGP
buying TJX
buying UNH

buying AOS
buying ACN
buying ADBE
buying AMD
buying AES
buying AFL
buying APD
buying ABNB
buying AKAM
buying ARE
buying ALLE
buying LNT
buying ALL
buying AMZN
buying AMCR
buying AEE
buying AEP
buying AXP
buying AIG
buying AMT
buying AWK
buying AMP
buying AME
buying APH
buying ADI
buying ANSS
buying AON
buying AAPL
buying AMAT
buying ACGL
buying ADM
buying ANET
buying AJG
buying AIZ
buying T
buying ATO
buying ADSK
buying AZO
buying AVB
buying AVY
buying AXON
buying BKR
buying BALL
buying BAC
buying BK
buying BBWI
buying BAX
buying BDX
buying BLK
buying BX
buying BA
buying BKNG
buying BSX
buying AVGO
buying BR
buying BRO
buying BLDR
buying BG
buying BXP
buying CDNS
buying CZR
buying CPB
buying COF
buying CAH
buying KMX
buying CCL
buying CARR
buying CBOE
buying CBRE
buying CDW
buying CE
buying COR
buying CNP
buying SCHW
buying CMG
buying CB
buying CI
buying CINF
buying CTAS
buying CSCO
buying C
buying CFG
buying CLX
buying CMS
buying KO
buying CTSH
buying CL
buying CMCSA
buying CAG
buying

buying AOS
buying ACN
buying ADBE
buying AMD
buying AES
buying AFL
buying A
buying AKAM
buying ARE
buying ALLE
buying LNT
buying ALL
buying AMZN
buying AMCR
buying AEE
buying AEP
buying AXP
buying AIG
buying AMT
buying AWK
buying AMP
buying AME
buying APH
buying ADI
buying ANSS
buying AON
buying AAPL
buying AMAT
buying ACGL
buying ADM
buying ANET
buying AJG
buying AIZ
buying T
buying ATO
buying ADSK
buying AZO
buying AVY
buying AXON
buying BALL
buying BAC
buying BK
buying BBWI
buying BAX
buying BLK
buying BX
buying BA
buying BKNG
buying BSX
buying AVGO
buying BR
buying BRO
buying BLDR
buying BG
buying BXP
buying CDNS
buying CZR
buying CPB
buying COF
buying CAH
buying KMX
buying CCL
buying CARR
buying CBOE
buying CBRE
buying CDW
buying CE
buying COR
buying CNC
buying SCHW
buying CMG
buying CB
buying CINF
buying CTAS
buying CSCO
buying C
buying CFG
buying CLX
buying CMS
buying KO
buying CTSH
buying CL
buying CAG
buying ED
buying STZ
buying CEG
buying COO
buying CPRT
buying GLW
buying CTV

buying CF
buying SCHW
buying CMG
buying CB
buying CTAS
buying CSCO
buying C
buying CFG
buying CLX
buying CME
buying KO
buying CTSH
buying CL
buying CMCSA
buying CAG
buying STZ
buying CEG
buying COO
buying CPRT
buying GLW
buying CPAY
buying CSGP
buying COST
buying CRWD
buying CCI
buying CSX
buying DHR
buying DRI
buying DVA
buying DAY
buying DECK
buying DE
buying DELL
buying DAL
buying DXCM
buying DLR
buying DFS
buying DLTR
buying D
buying DPZ
buying DOV
buying DOW
buying DHI
buying DTE
buying EMN
buying ETN
buying EBAY
buying ECL
buying EW
buying EA
buying ELV
buying ETR
buying EPAM
buying EFX
buying EQIX
buying EVRG
buying ES
buying EXC
buying EXPE
buying EXPD
buying EXR
buying FFIV
buying FDS
buying FICO
buying FAST
buying FRT
buying FDX
buying FIS
buying FITB
buying FI
buying FTV
buying FOX
buying BEN
buying GRMN
buying IT
buying GE
buying GEHC
buying GEN
buying GNRC
buying GPC
buying GPN
buying GL
buying GDDY
buying GS
buying HIG
buying HCA
buying HPE
buying HLT
buying HOLX
buying H

...

buying AOS
buying ABT
buying ABBV
buying ACN
buying ADBE
buying AMD
buying AES
buying AFL
buying A
buying APD
buying ABNB
buying AKAM
buying ALB
buying ARE
buying ALGN
buying ALLE
buying LNT
buying ALL
buying MO
buying AMZN
buying AMCR
buying AEE
buying AEP
buying AXP
buying AIG
buying AMT
buying AWK
buying AMP
buying AME
buying AMGN
buying APH
buying ADI
buying AON
buying AAPL
buying AMAT
buying ADM
buying ANET
buying AJG
buying AIZ
buying T
buying ATO
buying ADSK
buying ADP
buying AZO
buying AVB
buying AVY
buying AXON
buying BALL
buying BAC
buying BK
buying BBWI
buying BAX
buying BDX
buying BBY
buying TECH
buying BIIB
buying BLK
buying BX
buying BA
buying BKNG
buying BSX
buying BMY
buying AVGO
buying BR
buying BRO
buying BLDR
buying BXP
buying CHRW
buying CDNS
buying CZR
buying CPT
buying CPB
buying COF
buying CAH
buying KMX
buying CCL
buying CARR
buying CTLT
buying CAT
buying CBRE
buying CDW
buying CE
buying COR
buying CNC
buying CNP
buying CF
buying CRL
buying SCHW
buying CVX
buyin

buying AOS
buying ABT
buying ABBV
buying ACN
buying AMD
buying AES
buying A
buying ABNB
buying AKAM
buying ALB
buying ARE
buying ALGN
buying ALLE
buying ALL
buying GOOGL
buying GOOG
buying AMZN
buying AMCR
buying AEP
buying AXP
buying AIG
buying AMT
buying AMP
buying AME
buying AMGN
buying APH
buying ADI
buying AAPL
buying AMAT
buying APTV
buying ADM
buying ANET
buying ATO
buying ADSK
buying ADP
buying AVB
buying AVY
buying AXON
buying BALL
buying BAC
buying BK
buying BBWI
buying BAX
buying BBY
buying TECH
buying BIIB
buying BLK
buying BX
buying BA
buying BKNG
buying BWA
buying BMY
buying AVGO
buying BLDR
buying BXP
buying CHRW
buying CDNS
buying CZR
buying CPT
buying CPB
buying COF
buying KMX
buying CCL
buying CARR
buying CTLT
buying CAT
buying CBRE
buying CDW
buying CE
buying CNP
buying CRL
buying SCHW
buying CVX
buying CMG
buying CI
buying CTAS
buying CSCO
buying C
buying CFG
buying CTSH
buying CMCSA
buying CAG
buying COO
buying GLW
buying CPAY
buying CTVA
buying CSGP
buying COST
bu

buying AOS
buying ABT
buying ABBV
buying ACN
buying ADBE
buying AMD
buying AES
buying A
buying APD
buying ABNB
buying AKAM
buying ALB
buying ARE
buying ALGN
buying ALLE
buying GOOGL
buying GOOG
buying AMZN
buying AMCR
buying AEP
buying AXP
buying AIG
buying AMT
buying AWK
buying AMP
buying AME
buying AMGN
buying APH
buying ADI
buying ANSS
buying APA
buying AAPL
buying AMAT
buying APTV
buying ANET
buying ATO
buying ADSK
buying ADP
buying AVB
buying AVY
buying AXON
buying BKR
buying BALL
buying BAC
buying BK
buying BBWI
buying BAX
buying BDX
buying BBY
buying TECH
buying BIIB
buying BLK
buying BX
buying BA
buying BKNG
buying BWA
buying BSX
buying BMY
buying AVGO
buying BR
buying BLDR
buying BXP
buying CDNS
buying CZR
buying CPT
buying CPB
buying COF
buying KMX
buying CCL
buying CARR
buying CTLT
buying CAT
buying CBRE
buying CDW
buying CE
buying CNP
buying CF
buying CRL
buying SCHW
buying CHTR
buying CVX
buying CMG
buying CI
buying CTAS
buying CSCO
buying C
buying CFG
buying CLX
buying CM

selling MMM
buying AOS
buying ABT
buying ABBV
buying ACN
buying AMD
buying AES
buying A
buying APD
buying ABNB
buying AKAM
buying ALB
buying ARE
buying ALGN
buying ALLE
buying GOOGL
buying GOOG
buying AMZN
buying AMCR
buying AEP
buying AXP
buying AIG
buying AMT
buying AWK
buying AMP
buying AME
buying AMGN
buying APH
buying ADI
buying ANSS
buying APA
buying AMAT
buying APTV
buying ANET
buying ATO
buying ADSK
buying ADP
buying AVB
buying AVY
buying AXON
buying BKR
buying BAC
buying BK
buying BBWI
buying BAX
buying BDX
buying BBY
buying TECH
buying BIIB
buying BLK
buying BX
buying BA
buying BKNG
buying BWA
buying BSX
buying BMY
buying AVGO
buying BR
buying BLDR
buying BXP
buying CHRW
buying CDNS
buying CZR
buying CPT
buying CPB
buying COF
buying KMX
buying CCL
buying CARR
buying CTLT
buying CAT
buying CBRE
buying CDW
buying CE
buying COR
buying CF
buying CRL
buying SCHW
buying CHTR
buying CVX
buying CMG
buying CI
buying CTAS
buying CSCO
buying C
buying CFG
buying CLX
buying CME
buying CTS

buying AOS
buying ABT
buying ABBV
buying ACN
buying AMD
buying AES
buying A
buying APD
buying AKAM
buying ALB
buying ARE
buying ALGN
buying ALLE
buying GOOGL
buying GOOG
buying AMZN
buying AXP
buying AIG
buying AMT
buying AMP
buying AME
buying AMGN
buying APH
buying ADI
buying ANSS
buying APA
buying AMAT
buying APTV
buying ANET
buying T
buying ADSK
buying AVB
buying AVY
buying AXON
buying BKR
buying BAC
buying BK
buying BBWI
buying BAX
buying BDX
buying BBY
buying TECH
buying BIIB
buying BLK
buying BX
buying BA
buying BKNG
buying BWA
buying AVGO
buying BR
buying BLDR
buying BXP
buying CHRW
buying CZR
buying CPT
buying COF
buying KMX
buying CCL
buying CARR
buying CTLT
buying CAT
buying CBRE
buying CDW
buying CE
buying CF
buying CRL
buying SCHW
buying CHTR
buying CVX
buying CI
buying CTAS
buying CSCO
buying C
buying CFG
buying CTSH
buying CMCSA
buying COP
buying CEG
buying COO
buying GLW
buying CPAY
buying CTVA
buying CSGP
buying COST
buying CTRA
buying CRWD
buying CSX
buying CMI
buying 

In [10]:
final_value = cerebro.broker.getvalue()
print(f"Final Portfolio Value: ${final_value:.2f}")

Final Portfolio Value: $123594.78
